In [ ]:
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContentSettings
import json
import datetime
import time

In [ ]:
azure_storage_connection_string = "your-azure-storage-account-connection-string"
container_name = "your-container=name"
blob_name = "opensky_data.json"
opensky_api_url = "https://opensky-network.org/api/states/all"

In [ ]:
columns = ["icao24", "callsign", "origin_country", "time_position",
           "last_contact", "longitude", "latitude", "baro_altitude",
           "on_ground", "velocity", "true_track", "vertical_rate",
           "sensors", "geo_altitude", "squawk", "spi", "position_source"]

In [ ]:
def fetch_data_and_upload():
    try:
        # Fetch data from OpenSky API
        response = requests.get(opensky_api_url)
        data = response.json()

        if "states" in data:
            states = data["states"]
            timestamp = datetime.datetime.utcnow().isoformat()

            # Create BlobServiceClient
            blob_service_client = BlobServiceClient.from_connection_string(azure_storage_connection_string)

            # Upload each record as a separate JSON file
            for state in states:
                # Prepare data with column names
                formatted_state = dict(zip(columns, state))

                # Convert data to JSON
                data_to_upload = formatted_state
                json_data = json.dumps(data_to_upload)

                # Generate a unique filename based on timestamp
                filename = f"{timestamp}_{state[0]}.json"  # Using 'icao24' as part of filename
                blob_client = blob_service_client.get_blob_client(container=container_name, blob=filename)

                # Upload JSON data to Blob Storage
                blob_client.upload_blob(json_data, overwrite=True, content_settings=ContentSettings(content_type='application/json'))

                print(f"Uploaded data to Azure Blob Storage for {state[0]} at {timestamp}")

        else:
            print("No aircraft data found.")

    except Exception as e:
        print(f"Error fetching or uploading data: {str(e)}")

In [ ]:
if __name__ == "__main__":
    while True:
        fetch_data_and_upload()
        time.sleep(1)